In [10]:
import sys
sys.path.insert(0,'/cis/home/leebc/Projects/Mouse_Histology/code/pytorch/') # add code directory to path
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os

In [11]:
def myshow(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayFromImage(img)
    spacing = img.GetSpacing()

    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]

        # the the number of components is 3 or 4 consider it an RGB image
        if c not in (3, 4):
            nda = nda[nda.shape[0] // 2, :, :]

    elif nda.ndim == 4:
        c = nda.shape[-1]

        if c not in (3, 4):
            raise RuntimeError("Unable to show 3D-vector Image")

        # take a z-slice
        nda = nda[nda.shape[0] // 2, :, :, :]

    xsize = nda.shape[1]
    ysize = nda.shape[0]

    # Make a figure big enough to accommodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * xsize / dpi, (1 + margin) * ysize / dpi

    plt.figure(figsize=figsize, dpi=dpi, tight_layout=True)
    ax = plt.gca()

    extent = (0, xsize * spacing[0], ysize * spacing[1], 0)

    t = ax.imshow(nda, extent=extent, interpolation=None)

    if nda.ndim == 2:
        t.set_cmap("gray")

    if(title):
        plt.title(title, fontdict = {'fontsize' : 200})

    plt.show()

In [12]:
def get_center(img):
    width, height, depth = img.GetSize()

    return img.TransformIndexToPhysicalPoint((int(np.ceil(width/2)),
                                          int(np.ceil(height/2)),
                                          int(np.ceil(depth/2))))

In [13]:
def resample(image, transform):
    reference_image = image
    interpolator = sitk.sitkLinear
    default_value = 0.0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)

In [14]:
# Function for grabbing the max dimensions from a set of images 
def get_max_dimensions(files):
    file_dimensions = []
    max1 = 0; max2 = 0;
    for file in files:
        # read file 
        itkimage = sitk.ReadImage(file)

        # get dimension
        dimension = itkimage.GetSize()
        file_dimensions.append(dimension)

        # get the maximum largest x and largest y dimensions of the imageset
        if dimension[0] > max1:
            max1 = dimension[0]
        if dimension[1] > max2:
            max2 = dimension[1]
            
    return max1, max2, file_dimensions

In [20]:
%%time
# Change filepath to get to where files are saved
filepath = '/cis/home/jtrieu/Documents/HannoverSMIResampledGray/'
os.chdir(filepath)
files = os.listdir(filepath)

# thresh = [(16,255), (20,255), (18,228), (10,255),
#           (14,255), (14,247), (17,255), (9,254),
#           (12,238), (14,252), (10,247), (14,248),
#           (10,246), (12,255), (10,244), (12,255),
#           (14,243), (11,255), (15,255), (42,244),
#           (17,217), (16,250), (21,253), (18,220),
#           (23,212), (13,232), (13,213), (12,233),
#           (14,227), (13,216), (14,223), (8,229),
#           (14,232), (7,209), (22,255), (10,248)]

modified_images = []

for i, file in enumerate(files):
    filename = str(i) + 'gray.tif'
    img = sitk.ReadImage(filename)
    print(filename)
 
    seg = sitk.BinaryThreshold(img,
                               lowerThreshold=255, upperThreshold=10,
                               #lowerThreshold=thresh[i][1], upperThreshold=thresh[i][0],
                               insideValue=1, outsideValue=0)
    image_mask = sitk.Mask(img,seg,outsideValue = 0, maskingValue= 1)
    
    #myshow(seg, "binary")
    #myshow(image_mask, "masking")
    
    # Save image to modified images
    modified_images.append(image_mask)
    
    # Save image to modified images
    modified_images.append(image_mask)

filepath = '/cis/home/jtrieu/Documents'
os.chdir(filepath)
factor = 0.125
xy_spacing = 1.0745e-6 / factor
z_spacing = 250e-6
spacing = [xy_spacing, xy_spacing, z_spacing]
volume_images = sitk.JoinSeries(modified_images)
sitk.WriteImage(volume_images, 'Hannover_SMI_edited_compiled.tif', True)

0gray.tif
1gray.tif
2gray.tif
3gray.tif
4gray.tif
5gray.tif
6gray.tif
7gray.tif
8gray.tif
9gray.tif
10gray.tif
11gray.tif
12gray.tif
13gray.tif
14gray.tif
15gray.tif
16gray.tif
17gray.tif
18gray.tif
19gray.tif
20gray.tif
21gray.tif
22gray.tif
23gray.tif
24gray.tif
25gray.tif
26gray.tif
27gray.tif
28gray.tif
29gray.tif
30gray.tif
31gray.tif
32gray.tif
33gray.tif
34gray.tif
35gray.tif
36gray.tif
37gray.tif
38gray.tif
39gray.tif
40gray.tif
41gray.tif
42gray.tif
43gray.tif
44gray.tif
45gray.tif
46gray.tif
47gray.tif
48gray.tif
49gray.tif
50gray.tif
51gray.tif
CPU times: user 10.5 s, sys: 2.01 s, total: 12.5 s
Wall time: 8 s
